In [0]:
tables_df = spark.sql("SHOW TABLES IN harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gbmartecoschema")
display(tables_df)

for row in tables_df.collect():
    table_name = row['tableName']
    df_name = f"{table_name}_df"
    globals()[df_name] = spark.table(f"harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gbmartecoschema.{table_name}")

In [0]:
display(addresses_df)

In [0]:
from pyspark.sql.functions import regexp_replace

addresses_df = addresses_df.withColumn('AddressLine1', (regexp_replace(regexp_replace(regexp_replace('AddressLine1', '\n', ', '),'_',' '),'-',' ')))
addresses_df = addresses_df.withColumn('AddressType', regexp_replace('AddressType', '_', ' '))
addresses_df = addresses_df.withColumn('State', regexp_replace('State', '_', ' '))
display(addresses_df)

In [0]:
for df_name in ['addresses_df', 'customers_df', 'orders_df', 'orders_items_df', 'payment_methods_df', 'payments_df', 'products_df', 'returns_df', 'shipping_tier_df', 'suppliers_df']:
    df = globals()[df_name]
    display(
        df.select([
            df[c].isNull().cast("int").alias(c)
            for c in df.columns
        ]).groupBy().sum()
    )

In [0]:
# in product -> if there is null values 
products_df = products_df.fillna({
    'Product_Rating': 0,
    'Number_of_product_ratings': 0,
    'Discounted_Price': 0,
    'Actual_Price': 0
})
display(products_df)

In [0]:
products_df = products_df.dropna(
    subset=[
        'Product_Rating',
        'Number_of_product_ratings',
        'Discounted_Price',
        'Actual_Price'
    ]
)
display(products_df)

In [0]:
from pyspark.sql.functions import mean

mean_values = products_df.select([
    mean('Product_Rating').alias('mean_Product_Rating'),
    mean('Number_of_product_ratings').alias('mean_Number_of_product_ratings'),
    mean('Discounted_Price').alias('mean_Discounted_Price'),
    mean('Actual_Price').alias('mean_Actual_Price')
]).collect()[0]

products_df = products_df.fillna({
    'Product_Rating': mean_values['mean_Product_Rating'],
    'Number_of_product_ratings': mean_values['mean_Number_of_product_ratings'],
    'Discounted_Price': mean_values['mean_Discounted_Price'],
    'Actual_Price': mean_values['mean_Actual_Price']
})

In [0]:
# in payment table full zero where there is null in GiftCardAmount & discountAMount
payments_df = payments_df.fillna({
    'GiftCardAmount': 0,
    'CouponAmount': 0
})

In [0]:
for df_name in ['addresses_df', 'customers_df', 'orders_df', 'orders_items_df', 'payment_methods_df', 'payments_df', 'products_df', 'returns_df', 'shipping_tier_df', 'suppliers_df']:
    df = globals()[df_name]
    display(
        df.select([
            df[c].isNull().cast("int").alias(c)
            for c in df.columns
        ]).groupBy().sum()
    )

In [0]:
# display the dupplicates values in each column
for df_name in ['addresses_df', 'customers_df', 'orders_df', 'orders_items_df', 'payment_methods_df', 'payments_df', 'products_df', 'returns_df', 'shipping_tier_df', 'suppliers_df']:
    df = globals()[df_name]
    pk = df.columns[0]
    display(
        df.groupBy(pk).count().where('count > 1').orderBy('count', ascending=False)
    )

In [0]:
catalog_name = "harshit_gupta_databricks_npmentorskool_onmicrosoft_com"

In [0]:
%sql
-- create a schema for silver layer
CREATE SCHEMA IF NOT EXISTS harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.addresses(
    AddressID STRING PRIMARY KEY,
    CustomerID STRING,
    AddressLine1 STRING,
    City STRING,
    State STRING,
    PinCode DOUBLE,
    AddressType STRING
)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.customers (
    CustomerID STRING PRIMARY KEY,
    FirstName STRING,
    LastName STRING,
    Email STRING,
    PhoneNumber BIGINT,
    DateOfBirth DATE,
    RegistrationDate DATE,
    PreferredPaymentMethodID STRING
);


In [0]:
%sql
ALTER TABLE harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.addresses
ADD CONSTRAINT fk_customer FOREIGN KEY (CustomerID) REFERENCES harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.customers(CustomerID);


In [0]:
# I want to convert the OrderDate & ShippingDate in dateand time Format with HH:mm:ss
orders_df = orders_df.withColumn('OrderDate', orders_df['OrderDate'].cast('date'))
orders_df = orders_df.withColumn('ShippingDate', orders_df['ShippingDate'].cast('date'))
orders_df = orders_df.withColumn('ExpectedDeliveryDate', orders_df['ExpectedDeliveryDate'].cast('date'))
orders_df = orders_df.withColumn('ActualDeliveryDate', orders_df['ActualDeliveryDate'].cast('date'))
display(orders_df)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.orders (
    OrderID STRING PRIMARY KEY,
    CustomerID STRING,
    OrderDate DATE,
    ShippingDate DATE,
    ExpectedDeliveryDate DATE,
    ActualDeliveryDate DATE,
    ShippingTierID STRING,
    SupplierID STRING,
    OrderChannel STRING
);


In [0]:
%sql
CREATE TABLE IF NOT EXISTS harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.order_items (
    OrderItemID STRING PRIMARY KEY,
    OrderID STRING,
    ProductID STRING,
    Quantity INT
    -- Foreign key constraints can be added later as needed
);

In [0]:
%sql
CREATE TABLE IF NOT EXISTS harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.payment_methods (
    PaymentMethodID STRING PRIMARY KEY,
    MethodName STRING
);

In [0]:
payments_df = payments_df.withColumn('PaymentDate', payments_df['PaymentDate'].cast('date'))
display(payments_df)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.payments (
    PaymentID STRING PRIMARY KEY,
    OrderID STRING,
    PaymentDate DATE,
    GiftCardUsage STRING,
    GiftCardAmount DOUBLE,
    CouponUsage STRING,
    CouponAmount DOUBLE,
    PaymentMethodID STRING
    -- Foreign key constraints can be added later as needed
);


In [0]:
# I want to convert product rating & number of product rating into float
products_df = products_df.withColumn('Product_Rating', products_df['Product_Rating'].cast('float'))
products_df = products_df.withColumn('Number_of_product_ratings', products_df['Number_of_product_ratings'].cast('float'))

# I want to convert discounted price & actual price into double removing commas and the rupee sign 
products_df = products_df.withColumn('Discounted_Price', regexp_replace('Discounted_Price', '[,₹]', '').cast('double'))
products_df = products_df.withColumn('Actual_Price', regexp_replace('Actual_Price', '[,₹]', '').cast('double'))

display(products_df)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.products (
    Product_ID STRING PRIMARY KEY,
    Product_Name STRING,
    Product_Category STRING,
    Product_Sub_Category STRING,
    Product_Rating FLOAT,
    Number_of_product_ratings FLOAT,
    Discounted_Price DOUBLE,
    Actual_Price DOUBLE
);


In [0]:
%sql
CREATE TABLE IF NOT EXISTS harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.returns (
    OrderID STRING,
    Return_reason STRING
    -- You can add a primary key or additional columns as needed
);

In [0]:
%sql
CREATE TABLE IF NOT EXISTS harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.shipping_tier(
    ShippingTierID STRING PRIMARY KEY,
    TierName STRING,
    Cost_Rs INT
);
    
CREATE TABLE IF NOT EXISTS harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.suppliers(
    SupplierID STRING PRIMARY KEY,
    SupplierName STRING,
    SupplierEmail STRING,
    City STRING
);

In [0]:
%sql
ALTER TABLE harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.customers
ADD CONSTRAINT fk_payment_method FOREIGN KEY (PreferredPaymentMethodID) REFERENCES harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.payment_methods(PaymentMethodID);

In [0]:
%sql
ALTER TABLE harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.orders
ADD CONSTRAINT fk_customer_order FOREIGN KEY (CustomerID) REFERENCES harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.customers(CustomerID);

ALTER TABLE harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.orders
ADD CONSTRAINT fk_shipping_tier FOREIGN KEY (ShippingTierID) REFERENCES harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.shipping_tier(ShippingTierID);

ALTER TABLE harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.orders
ADD CONSTRAINT fk_supplier FOREIGN KEY (SupplierID) REFERENCES harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.suppliers(SupplierID);

In [0]:
%sql
ALTER TABLE harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.payments
ADD CONSTRAINT fk_order_payment FOREIGN KEY (OrderID) REFERENCES harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.orders(OrderID);

ALTER TABLE harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.payments
ADD CONSTRAINT fk_payment_method_order FOREIGN KEY (PaymentMethodID) REFERENCES harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.payment_methods(PaymentMethodID);

In [0]:
%sql
ALTER TABLE harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.returns
ADD CONSTRAINT fk_returns FOREIGN KEY (OrderID) REFERENCES harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.orders(OrderID);

In [0]:
%sql
ALTER TABLE harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.order_items
ADD CONSTRAINT fk_order_items_order FOREIGN KEY (OrderID) REFERENCES harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.orders(OrderID);

ALTER TABLE harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.order_items
ADD CONSTRAINT fk_order_items_product FOREIGN KEY (ProductID) REFERENCES harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.products(Product_ID);

In [0]:
#load the data into harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.{table} from {table}_df
for row in tables_df.collect():
    table_name = row['tableName']
    df_name = table_name + '_df'
    if table_name == 'orders_items':
        table_name = 'order_items'
    globals()[df_name].write.mode('overwrite').format('delta').insertInto(f'harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.{table_name}')

In [0]:
for row in tables_df.collect():
    table_name = row['tableName']
    if table_name == 'orders_items':
        table_name = 'order_items'
    spark.sql(f"ALTER TABLE harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gcs_silver.{table_name} SET TBLPROPERTIES (delta.enableChangeDataFeed = true)")